In [1]:
from Guwahati import nodeinfo, instrumentation_asset_info

inst_info, asset_info = instrumentation_asset_info



In [3]:
from nwater.nw_hierarchy import NW_hierarchy_from_Guwahati

# Test the function
roots = NW_hierarchy_from_Guwahati(nodeinfo, inst_info, asset_info)

print(roots)

69 has asset 79
70 has asset 87
71 has asset 88
73 has asset 80
75 has asset 82
76 has asset 90
77 has asset 96
78 has asset 83
79 has asset 84
80 has asset 91
82 has asset 85
83 has asset 93
84 has asset 86
85 has asset 94
97 has child 99
99 has child 100
97 has child 101
101 has child 102
101 has child 103
101 has child 104
100 has instrument 69
100 has instrument 70
100 has instrument 71
102 has instrument 73
102 has instrument 75
102 has instrument 76
102 has instrument 77
102 has instrument 78
103 has instrument 79
104 has instrument 85
104 has instrument 83
104 has instrument 82
104 has instrument 80
104 has instrument 84
[nw_node(id=97, name='Nijeshwari PWSS', type='location', subnodes=[nw_node(id=99, name='Abstraction', type='water_abstraction', subnodes=[nw_node(id=100, name='Source', type='source_module', subnodes=[nw_instrument(id=69, name='Borewell level', type='level', subnodes=[nw_asset(id=79, name='D1004901122', type='asset', subnodes=[], prod_code='FMX21', prod_name='Wa

In [16]:
def count_nodes_in_hierarchy(node, counts=None):
    """
    Count different types of nodes in the hierarchy.
    
    Args:
        node: The nw_node to count from
        counts: Dictionary to accumulate counts
    
    Returns:
        Dictionary with counts of each node type
    """
    if counts is None:
        counts = {
            'locations': 0,
            'applications': 0,
            'modules': 0,
            'instruments': 0,
            'assets': 0
        }
    
    # Count current node
    if hasattr(node, 'tag') and hasattr(node, 'primary_val_key'):
        # This is an nw_instrument
        counts['instruments'] += 1
    elif hasattr(node, 'prod_code'):
        # This is an nw_asset
        counts['assets'] += 1
    else:
        # This is a regular nw_node
        if node.type == 'location':
            counts['locations'] += 1
        elif node.type in ['water_abstraction', 'water_distribution', 'effluent_discharge']:
            counts['applications'] += 1
        elif 'module' in node.type:
            counts['modules'] += 1
    
    # Recursively count subnodes
    if hasattr(node, 'subnodes') and node.subnodes:
        for subnode in node.subnodes:
            count_nodes_in_hierarchy(subnode, counts)
    
    return counts

def pretty_print_nw_hierarchy(node, level=0):
    """
    Pretty print the NW hierarchy starting from a root node.
    
    Args:
        node: The nw_node to print (location, application, module, instrument, or asset)
        level: Current depth level for indentation (starts at 0)
    """
    # Create indentation (5 spaces per level)
    indent = "     " * level
    
    # Print current node with type information
    if hasattr(node, 'tag') and hasattr(node, 'primary_val_key'):
        # This is an nw_instrument
        primary_key = node.primary_val_key or "None"
        print(f"{indent}🔧 {node.name} (ID: {node.id}, Type: {node.type}, Primary: {primary_key})")
    elif hasattr(node, 'prod_code'):
        # This is an nw_asset
        print(f"{indent}📦 {node.name} (ID: {node.id}, Product: {node.prod_code})")
    else:
        # This is a regular nw_node (location, application, module)
        type_emoji = {
            'location': '🏢',
            'water_abstraction': '💧',
            'water_distribution': '🚰',
            'effluent_discharge': '🌊',
            'source_module': '🏔️',
            'storage_module': '🏪',
            'disinfection_module': '🧽',
            'outlet_module': '🚪',
            'inlet_module': '🚪',
            'transfer_module': '🔄',
            'quality_control_module': '🔬'
        }.get(node.type, '📍')
        
        print(f"{indent}{type_emoji} {node.name} (ID: {node.id}, Type: {node.type})")
    
    # Recursively print all subnodes
    if hasattr(node, 'subnodes') and node.subnodes:
        for subnode in node.subnodes:
            pretty_print_nw_hierarchy(subnode, level + 1)

def print_hierarchy_summary(roots):
    """
    Print a summary of the entire hierarchy showing counts of each node type.
    
    Args:
        roots: List of root nodes to analyze
    """
    total_counts = {
        'locations': 0,
        'applications': 0,
        'modules': 0,
        'instruments': 0,
        'assets': 0
    }
    
    # Count nodes from all roots
    for root in roots:
        root_counts = count_nodes_in_hierarchy(root)
        for key in total_counts:
            total_counts[key] += root_counts[key]
    
    print("=" * 50)
    print("📊 HIERARCHY SUMMARY")
    print("=" * 50)
    print(f"🏢 Locations:      {total_counts['locations']:3d}")
    print(f"💧 Applications:   {total_counts['applications']:3d}")
    print(f"📦 Modules:        {total_counts['modules']:3d}")
    print(f"🔧 Instruments:    {total_counts['instruments']:3d}")
    print(f"📦 Assets:         {total_counts['assets']:3d}")
    print("-" * 50)
    total_nodes = sum(total_counts.values())
    print(f"📋 Total Nodes:    {total_nodes:3d}")
    print("=" * 50)

# Test the pretty print function with your hierarchy
print("=== NW Hierarchy Pretty Print ===")
for root in roots:
    pretty_print_nw_hierarchy(root)
    print()  # Add empty line between different root nodes

# Print summary
print_hierarchy_summary(roots)

=== NW Hierarchy Pretty Print ===
🏢 Nijeshwari PWSS (ID: 97, Type: location)
     💧 Abstraction (ID: 99, Type: water_abstraction)
          🏔️ Source (ID: 100, Type: source_module)
               🔧 Borewell level (ID: 69, Type: level, Primary: groundwater_level)
                    📦 D1004901122 (ID: 79, Product: FMX21)
               🔧 Power Mains (ID: 70, Type: power, Primary: total_kwh)
                    📦 2407811548 (ID: 87, Product: Energy meter)
               🔧 ABS-Pump (ID: 71, Type: pump, Primary: abs_pump_on)
                    📦 Guwahati_ABS-Pump01 (ID: 88, Product: Pump)
     🚰 Distribution (ID: 101, Type: water_distribution)
          🧽 Treated water (ID: 102, Type: disinfection_module)
               🔧 Disinfect Transmitter (ID: 73, Type: undefined, Primary: None)
                    📦 L70E7D05G00 (ID: 80, Product: CM442)
               🔧 Chlorine (ID: 75, Type: analysis, Primary: None)
                    📦 X3048005C00 (ID: 82, Product: CCS51E)
               🔧 Chlori